# Kliment Mamykin, UNI 2770
## Algorythms for Data Science, Homework 1

### Problem 1(a)

**Process:** restate the algorythm, establish loop invariant and prove the correctness by showing that the loop invariant holds at the initialization before the loop, after each iteration of the loop, and after the termination.

**Insight:** a polynomial of order $n$ $$
p ( x ) = a_{0} + a_{1} x + a_{2} x^{2} + \ldots + a_{n} x^{n} = \sum_{k=0}^{n} {a_{k}x^k}
$$ can be re-written as 

\begin{equation}
p ( x ) = a_{0} + (a_{1} + (a_{2} + \ldots (a_{n-1} + (a_{n})x) \ldots )x)x
\end{equation}


**Loop invariant**: At the start of each $i$ loop, $z$ contains the value of polynomial of order $n-(i+1)$ with coefficients $A[i+1..n]$. Mathematically 
$$
z_i = \sum_{k=0}^{n-(i+1)} {a_{k+i+1}x^k}
$$

**Initialization:** We need to prove that for $i = n-1$ the initialization value of $z=A[n]$ satisfies the loop invariant. 

$z$ contains the value of the polinomial of order $n - i - 1 = n - (n - 1) - 1 = 0$ with coefficients $A[i+1..n] = A[(n-1)+1..n] = A[n]$. Therefor initialization of $z = A[n]$ satisfies the loop invariant. 

$$
z_{n-1} = \sum_{k=0}^{n-(n-1+1)} {a_{k+(n-1)+1}x^k} = a_{n}x^0 = a_n
$$

**Maintenance:** At each iteration of the loop $z$ is assigned a new value to be used on the next loop iteration $z_{i-1}$ (loop is counting down) based on the value before the loop iteration $z_i$. At a particular interation $i$ we have 

$$
z_{i-1} = a_i + z_{i} x = a_i + (\sum_{k=0}^{n-(i+1)} {a_{k+i+1}x^k}) x = \sum_{k=0}^{n-i} {a_{k+i}x^k}  = \sum_{k=0}^{n-((i-1)+1)} {a_{k+(i-1)+1}x^k}
$$

After the loop completes, $z_{i-1}$ for the next iteration of the loop satisfies teh invariant.

**Termination:** Eventually $i$ will be assigned the value $-1$, the condition of the _for_ loop fails and loop terminates. At this point $z$ contain the value of the last loop iteration at $i=0$. We need to prove that at this point $z = \sum_{k=0}^{n} {a_{k}x^k}$ based on the invariant.

$$
z = \sum_{k=0}^{n-(-1+1)} {a_{k+(-1)+1}x^k} = \sum_{k=0}^{n} {a_{k}x^k}
$$

### Problem 1(b)

Horner's rule uses $n$ multiplications and $n$ additions. Each iteration of the loop from $n-1$ to $0$ ($n$ iterations performs one addition and one multiplication.

Even for a polynomial with large $n$ and all lower terms with 0 coefficients $A = [0,0,\ldots,0,a_n]$ the naive calculation could shirtcut multiplication by 0, but the check for 0 for each coefficient would still take constant time and the naive implementation will not be more efficient then Horner's rule.

### Problem 2(a)
